In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 53.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import spacy

# Load the dataset
file_path = '/content/sampled_dataECBGoldSilver.csv'
data = pd.read_csv(file_path)

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

def extract_triples(text):
    """
    Extract subject-relation-object triples from a given text using spaCy dependency parsing.
    """
    if not isinstance(text, str):
        return []

    doc = nlp(text)
    triples = []
    for sent in doc.sents:
        subject = ''
        relation = ''
        obj = ''
        for token in sent:
            if token.dep_ in ("nsubj", "nsubjpass") and subject == '':
                subject = token.lemma_  # Use lemma to normalize the word
            elif token.dep_ == "ROOT":
                relation = token.lemma_  # Use lemma to normalize the word
            elif token.dep_ in ("attr", "dobj"):
                obj = token.lemma_  # Use lemma to normalize the word
        if subject and relation and obj:
            triples.append((subject, relation, obj))
    return triples

def compare_facts(generated_triples, reference_triples):
    correct_facts = 0
    for gen_triplet in generated_triples:
        if gen_triplet in reference_triples:
            correct_facts += 1
    return correct_facts, len(generated_triples)

def fact_score(generated_text, reference_text):
    generated_triples = extract_triples(generated_text)
    reference_triples = extract_triples(reference_text)

    correct_facts, total_facts = compare_facts(generated_triples, reference_triples)

    if total_facts == 0:
        return 0
    return correct_facts / total_facts

# Convert relevant columns to string type to ensure compatibility with spaCy
data['amalREC_Test_Sentence'] = data['amalREC_Test_Sentence'].astype(str)
data['Gold_Sentence'] = data['Gold_Sentence'].astype(str)

# Calculate Fact Scores for the dataset
data['Fact_Score'] = data.apply(lambda row: fact_score(row['amalREC_Test_Sentence'], row['Gold_Sentence']), axis=1)

# Display the first few rows of the dataset with Fact Scores
data[['amalREC_Test_Sentence', 'Gold_Sentence', 'Fact_Score']].head()

,amalREC_Test_Sentence,Gold_Sentence,Fact_Score
0,"Renowned for his leadership prowess, the Grand...","The distinguished group, Grande Armée had the ...",0.0
1,The renowned Chislehurst and Sidcup Grammar Sc...,Chislehurst and Sidcup Grammar School a promin...,0.0
2,Nestled amidst the picturesque landscapes of t...,Réal can be found within the department of Pyr...,0.0
3,As the rector of Kremenchuk Mykhailo Ostrohrad...,Kremenchuk Mykhailo Ostrohradskyi National Uni...,0.0
4,"Kraai River, originating in the rolling hills ...",Kraai River is an important source of water fo...,0.0


In [1]:
import spacy
import pandas as pd

# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")

def extract_triples(text):
    """
    Extract subject-relation-object triples using SpaCy with a custom pipeline.
    """
    doc = nlp(text)
    triples = []
    for sent in doc.sents:
        subject = ''
        relation = ''
        obj = ''
        for token in sent:
            if token.dep_ in ("nsubj", "nsubjpass") and subject == '':
                subject = token.lemma_  # Use lemma to normalize the word
            elif token.dep_ == "ROOT":
                relation = token.lemma_  # Use lemma to normalize the word
            elif token.dep_ in ("attr", "dobj"):
                obj = token.lemma_  # Use lemma to normalize the word
        if subject and relation and obj:
            triples.append((subject, relation, obj))
    return triples

def compare_facts(generated_triples, reference_triples):
    correct_facts = 0
    for gen_triplet in generated_triples:
        if gen_triplet in reference_triples:
            correct_facts += 1
    return correct_facts, len(generated_triples)

def fact_score(generated_text, reference_text):
    generated_triples = extract_triples(generated_text)
    reference_triples = extract_triples(reference_text)

    correct_facts, total_facts = compare_facts(generated_triples, reference_triples)

    if total_facts == 0:
        return 0
    return correct_facts / total_facts

# Load the dataset
file_path = '/content/sampled_dataECBGoldSilver.csv'
data = pd.read_csv(file_path)

# Convert relevant columns to string type to ensure compatibility with SpaCy
data['amalREC_Test_Sentence'] = data['amalREC_Test_Sentence'].astype(str)
data['Gold_Sentence'] = data['Gold_Sentence'].astype(str)

# Calculate Fact Scores for the dataset
data['Fact_Score'] = data.apply(lambda row: fact_score(row['amalREC_Test_Sentence'], row['Gold_Sentence']), axis=1)

# Calculate the final average Fact Score across all sentences
average_fact_score = data['Fact_Score'].mean()

# Display the final average Fact Score
average_fact_score


0.01